In [51]:
import pandas as pd
import json
import os
import re

### adjust your analysis file and paths

In [35]:
#myfilepath = "enter/path/here"
myfilepath = r"F5_macrophage and SGC confocal/"
import sys
sys.path.append(myfilepath)  
import analysis_perDRG_F5_seperate_images #adjust file here

In [37]:
#open all json result files
#adjust files accordingly 
with open (myfilepath + "F5_W5_M.json") as f:
    results_w5_m = json.load (f)
    
with open (myfilepath + "F5_W1_M.json") as f:    
    results_w1_m = json.load (f)
      
with open (myfilepath + "F5_W1_F.json") as f:
    results_w1_f = json.load (f)
    
with open (myfilepath + "F5_W5_F.json") as f:
    results_w5_f = json.load (f)


## prepare data, IDs and functions

In [39]:
#set data in groups

w5_f_il = analysis_perDRG_F5_seperate_images.GroupData (results_w5_f, "IL" )
w5_f_cl = analysis_perDRG_F5_seperate_images.GroupData (results_w5_f, "CL" )
w1_f_il = analysis_perDRG_F5_seperate_images.GroupData (results_w1_f, "IL" )
w1_f_cl = analysis_perDRG_F5_seperate_images.GroupData (results_w1_f, "CL" )

w5_m_il = analysis_perDRG_F5_seperate_images.GroupData (results_w5_m, "IL")
w5_m_cl = analysis_perDRG_F5_seperate_images.GroupData (results_w5_m, "CL")
w1_m_il = analysis_perDRG_F5_seperate_images.GroupData (results_w1_m, "IL")
w1_m_cl = analysis_perDRG_F5_seperate_images.GroupData (results_w1_m, "CL")

In [43]:
paths_1wf = w1_f_il.paths
paths_5wf = w5_f_il.paths
paths_1wm = w1_m_il.paths
paths_5wm = w5_m_il.paths

In [45]:
def find_IDs (path): 
    parts = path.split(os.sep)
    rat = next((p for p in parts if re.match(r"^R\w+", p)), None)
    return f"{rat}"

In [47]:
#for excel export: set rat IDs as they are in the json files
IDs_1wf = [find_IDs(p) for p in paths_1wf if find_IDs(p) is not None]
IDs_5wf = [find_IDs(p) for p in paths_5wf if find_IDs(p) is not None]
IDs_1wm = [find_IDs(p) for p in paths_1wm if find_IDs(p) is not None]
IDs_5wm = [find_IDs(p) for p in paths_5wm if find_IDs(p) is not None]

In [61]:
def append_dct_confocal(IDs_L, gender, condition, group_il, group_cl, params):
    
    for i, rat_id in enumerate(IDs_L):
        #print(rat_id)  
        for x in range (6):
            #print (x)
            dct["ID"].append(rat_id)
            #print ("ID:" , rat_id)
            dct["gender"].append(gender)
            dct["condition"].append(condition)
        
        for pn, pname_full, label in params:
            il_vals = getattr(group_il, pname_full)[i]
            cl_vals = getattr(group_cl, pname_full)[i]

            for img_idx in range(6):
                dct[f"{pn} name"].append(label)
                #print (f"{pn} name" ,label)
                dct[f"{pn} IL"].append(il_vals[img_idx])
                #print (f"{pn} IL",il_vals[img_idx])
                dct[f"{pn} CL"].append(cl_vals[img_idx])
                #print (f"{pn} CL" ,cl_vals[img_idx])
            #print ("\n")

## define parameters and get files 

In [63]:
#adjust your export parameters
saveto = "Excel files/F5/"
figure = "Thesis --"
name = "inv and dil pct iba1 and fabp7"

params = [ ("P1", "inv_pct_iba1_allrats", "Iba1 per inv")
          ,("P2", "dil1_pct_iba1_allrats", "Iba1 per dil1")
          ,("P3", "dil2_pct_iba1_allrats", "Iba1 per dil2")
        
          ,("P4", "inv_pct_fabp7_allrats", "fabp7 per inv")
          ,("P5", "dil1_pct_fabp7_allrats", "fabp7 per dil1")
          ,("P6", "dil2_pct_fabp7_allrats", "fabp7 per dil2")

         #,("P7", "gs_iba1_proximity_allrats", "Iba1 per GS - Proximity")

         ]

#create empty dictionary
dct = {"ID" : [], "gender" : [], "condition": [] 
       ,"P1 name": [], "P1 IL": [], "P1 CL" : []
       ,"P2 name": [], "P2 IL": [], "P2 CL" : []
       ,"P3 name": [], "P3 IL": [], "P3 CL" : []
       ,"P4 name": [], "P4 IL": [], "P4 CL" : []
       ,"P5 name": [], "P5 IL": [], "P5 CL" : []
       ,"P6 name": [], "P6 IL": [], "P6 CL" : []
       #,"P7 name": [], "P7 IL": [], "P7 CL" : []
      }

#ID_L, gender, condition, group_il, group_cl, p1_name, p1
append_dct_confocal (IDs_1wm, "M", "1W", w1_m_il, w1_m_cl, params)
append_dct_confocal (IDs_1wf, "F", "1W", w1_f_il, w1_f_cl, params)
append_dct_confocal (IDs_5wm, "M", "5W", w5_m_il, w5_m_cl, params)
append_dct_confocal (IDs_5wf, "F", "5W", w5_f_il, w5_f_cl, params)
df = pd.DataFrame (dct)
#df_mean = group_by_animal (df)
df.to_excel (saveto + figure + name +".xlsx")

In [59]:
count_dict = {key: len(value) for key, value in dct.items()}

print(count_dict)


{'ID': 96, 'gender': 96, 'condition': 96, 'P1 name': 96, 'P1 IL': 96, 'P1 CL': 96, 'P2 name': 96, 'P2 IL': 96, 'P2 CL': 96, 'P3 name': 96, 'P3 IL': 96, 'P3 CL': 96, 'P4 name': 192, 'P4 IL': 192, 'P4 CL': 192, 'P5 name': 0, 'P5 IL': 0, 'P5 CL': 0, 'P6 name': 96, 'P6 IL': 96, 'P6 CL': 96}
